In [ ]:
import pandas as pd
import altair as alt

In [ ]:
ambry = '/Users/ivan/Downloads/Ambry_BARD1.xlsx'
sge_data = '../Data/20250122_BARD1_SGEscores_wAAsub.xlsx'
gnomad = '../Data/20240905_BARD1_gnomADv4.1.0_SNVs.xlsx'

In [ ]:
def read_data(ambry, gnomad, sge):
    ambry = pd.read_excel(ambry)
    ambry = ambry[['Gene', 'c_variant', 'p_variant', 'Classification']]

    
    sge = pd.read_excel(sge)
    sge = sge[['chrom', 'target','AAsub', 'Consequence', 'snv_score']]

    
    gnomad = pd.read_excel(gnomad)
    gnomad = gnomad[['Chromosome', 'Position', 'Reference', 'Alternate', 'Allele Frequency']]

    return ambry, sge, gnomad

In [ ]:
def main():
    ambry_data, sge_data, gnomad_data = read_data(ambry, gnomad, sge_data)
    

In [ ]:
main()